# **The Art of Holding a Lead** 
##### By Kento Abeywardane and Matthew Paull

## **Overview**
#### Our objective was to analyze defensive trends towards the end of the game. Specifically we thought it would be interesting to look at situations where a team is leading by 4-8 points with 4 minutes or less left in the fourth quarter. Our goal was to find defensive trends that contributed to a win. 
#### We split up the data between the teams that held their 4-8 point lead to win the game and the teams that blew the lead to lose the game. Then looking at the number of pass rushers from both situations we found that the teams who held their lead used an average of 3.95 pass rushers during their lead within the last 4 minutes, while the teams that blew the lead used an average of 4.12 pass rushers. 
#### When comparing the same teams defense from the rest of the game we found the average number of pass rushers before 4 minutes left was 4.07 for the teams that held their lead, and 4.1 for the the teams that blew their lead. While these are essentually equal, it is interesting to see that the teams who blew their lead tended to slightly increase their pass rush, while the teams who kept their lead to win tended to slightly decrease their pass rush. We found a similar trend for the number of defenders in the box as well.
#### Next we ran a two-sample T test on the number of pass rushers for the teams who held and blew their leads with under 4 minutes left in the game and got a P-value of 0.17. Since this is a very specific situation we only found 77 games that met our criteria, therefore if this same analysis was run over several seasons to gather more data it might be possible to find data with significance. 
#### Based off of our analysis, we would reccomend to teams who are leading by 4-8 points towards the end of the game that they should tend towards a consevative pass rush to win the game. 

## **Import Libraries**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
sns.set_style('whitegrid')

## **Import Games and Plays Dataframes**



In [ ]:
games = pd.read_csv('../input/nfl-big-data-bowl-2021/games.csv')
plays = pd.read_csv('../input/nfl-big-data-bowl-2021/plays.csv')

## **Data Cleaning**
#### Create a new Dataframe (final) from 'Plays' that has all of the plays from 4 minutes or less in the 4th quarter, and where the score differential is within 4-8 points

In [ ]:
# make 'gameClock' values to a datetime value from string
plays['gameClock'] = pd.to_datetime(plays['gameClock'],format= '%H:%M:%S' ).dt.time

# Add the Team Names into plays
def gameidtoT(i, col):
    return games[games['gameId'] == i][col].values[0]

plays['homeTeamAbbr'] = plays['gameId'].apply(gameidtoT, col='homeTeamAbbr')
plays['visitorTeamAbbr'] = plays['gameId'].apply(gameidtoT, col='visitorTeamAbbr')

# Add the team name thats leading for each play in plays
winning_team_list = list()
for i in range(0, len(plays)):
  home_score = plays.loc[i, 'preSnapHomeScore']
  away_score = plays.loc[i, 'preSnapVisitorScore']
  home_team = plays.loc[i, 'homeTeamAbbr']
  away_team = plays.loc[i, 'visitorTeamAbbr']
  if home_score > away_score:
    winning_team = home_team
  elif away_score > home_score:
    winning_team = away_team
  else:
    winning_team = 'Tied'
  winning_team_list.append(winning_team)

winning_team_array = np.array(winning_team_list)
plays['TeamWinning'] = winning_team_array

playsq4 = plays[plays['quarter'] == 4] # plays from the fourth quarter

import datetime
fourmin = datetime.time(4, 0, 0)
playsq4_4min = playsq4[playsq4['gameClock'] <= fourmin]
difference = abs(playsq4_4min['preSnapHomeScore'] - playsq4_4min['preSnapVisitorScore'])

# Set final to be the data frame with all plays where the score is in [4, 8] and there is <= 4 min left in the game
final = playsq4_4min[(difference <= 8) & (difference >=4)]

#### Drop the plays where the winning team has the ball (resulting in a dataframe where the winning team is on defense)

In [ ]:
final = final.drop(final[final['TeamWinning'] == final['possessionTeam']].index)
final = final.reset_index(drop=True)
final = final.set_index(['gameId','playId'])

#### Since there was no data on who won each game, we checked each of the 77 games in final for the winner. This data was collected in an excel sheet which we read into Google Collab (tyler_data_frame)
#### Add a column (teamWin) to final which represents the winner of each game

In [ ]:
tyler_data_frame = pd.read_csv('../input/tyler-data-frame/tyler_data_frame - tyler_data_frame.csv')
teamWin = tyler_data_frame['TeamWin']

# Add teamWin as a collumn in final
final = final.reset_index()
def tylertoW(i):
    return tyler_data_frame[tyler_data_frame['gameId'] == i]['TeamWin'].values[0]

final['teamWin'] = final['gameId'].apply(tylertoW)

#### Add a column (heldOrBlown) to final which signifies whether the team that was winning by 4-8 points originally held on to their lead or blew it

In [ ]:
# The first play of each game in final
firstplayof4m = final.reset_index().set_index(['gameId']).groupby('gameId')['playId'].min()

# See if the winning team going into 4 minutes held or blew their lead
def heldorblown(i):
    fpof = firstplayof4m[firstplayof4m.index == i].loc[i]
    #firstrow = final.loc[(i, fpof)]
    firstrow = final[(final['gameId'] == i) & (final['playId'] == fpof)]
    if firstrow['TeamWinning'].values[0] == firstrow['teamWin'].values[0]:
        return 'held'
    else:
        return 'blown'
    
final['heldOrBlown'] = final['gameId'].apply(heldorblown)

final = final.set_index(['gameId','playId'])

#### Identify situations where the initial winning team blew the lead, but then scored again to win (This should count as a blown lead from the defensive perspective). Switch to blown in the heldOrBlown column

In [ ]:
check = final.reset_index()
gameIds = np.unique(check['gameId'])
change_to_blown = []
for i in range(0, 77): # 77 games
  count = 0
  first_play = firstplayof4m.loc[gameIds[i]]
  first_row = final.loc[(gameIds[i], first_play)]
  team_winning_initially = first_row['TeamWinning']
  plays_wanted = playsq4_4min[playsq4_4min['gameId'] == gameIds[i]]
  index_of_plays_for_this_game = plays_wanted[plays_wanted['gameId'] ==  gameIds[i]]['playId'].values
  index_of_plays_for_this_game = index_of_plays_for_this_game[index_of_plays_for_this_game >= first_play]
  for j in range(0, len(index_of_plays_for_this_game)):
    individual_play_row = plays_wanted.loc[plays_wanted['playId'] == index_of_plays_for_this_game[j]]
    current_team_winning = individual_play_row['TeamWinning']
    if current_team_winning.values[0] != team_winning_initially: # if the losing team comes back
      count = count + 1
  if count > 0:
    if team_winning_initially == first_row['teamWin']: # if the losing team came back but the original team won the game
       change_to_blown.append(gameIds[i])
for i in change_to_blown:
    final.loc[(i),'heldOrBlown'] = 'blown'

#### Define two new dataframes (held and blown) that represent games where teams held their lead and games where teams blew there lead

In [ ]:
held = final[final['heldOrBlown'] == 'held'].reset_index() 
blown = final[final['heldOrBlown'] == 'blown'].reset_index()

In [ ]:
sns.scatterplot('defendersInTheBox','offensePlayResult',hue='heldOrBlown',data=final)
plt.show()

In [ ]:
sns.scatterplot('numberOfPassRushers','defendersInTheBox',hue='heldOrBlown',data=final)
plt.show()

## **Data Analysis**
#### Define functions that give the percentage of pass rush selections per game and DataFrame

In [ ]:
# each index of the array reprensents number of pass rushers
normal = final.reset_index()
def percentOfPassRushersPerGame(gameid, df):
  pass_rushers = np.array(df[df['gameId'] == gameid]['numberOfPassRushers'])
  number_of_plays = len(pass_rushers)
  pass_rush_list = []
  for i in range(0, 8):
    pass_rush_list.append(len(pass_rushers[pass_rushers == i]) / number_of_plays)
  return np.array(pass_rush_list)

def percentOfPassRushersByDataFrame(dataframe, df):
  final_array = np.zeros(8)
  game_ids = np.unique(dataframe['gameId'])
  for i in range(0, len(game_ids)):
    final_array = final_array + percentOfPassRushersPerGame(game_ids[i], df)
  final_array = final_array / len(game_ids)
  return final_array



#### Define a function that gives the average number of pass rushers per game by DataFrame

In [ ]:
def averagePassRushersPerGameByDF(dataframe, df):
  lister = []
  game_ids = np.unique(dataframe['gameId'])
  for i in range(0, len(game_ids)):
    percents = percentOfPassRushersPerGame(game_ids[i], df)
    average = 0
    for i in range(0, 8):
      average = average + (percents[i] * i)
    lister.append(average)
  return np.array(lister)

#### Compute a T-test on the average number of pass rushers for held-lead games and blown-lead games

In [ ]:
held_average = averagePassRushersPerGameByDF(held, normal)
blown_average = averagePassRushersPerGameByDF(blown, normal)
stats.ttest_ind(held_average, blown_average, equal_var=False)

#### Define a function that gives the average number of defenders in the box per game by DataFrame

In [ ]:
def averageDefendersInTheBoxPerGameByDF(dataframe):
  lister = []
  gameids = np.unique(dataframe['gameId'])
  for i in range(0, len(gameids)):
    average_defenders_in_box = dataframe[dataframe['gameId'] == gameids[i]]['defendersInTheBox'].mean()
    lister.append(average_defenders_in_box)
  return(np.array(lister))

#### Compute a T-test on the average number of defenders in the box for held-lead games and blown-lead games

In [ ]:
held_defenders_in_box = averageDefendersInTheBoxPerGameByDF(held)
blown_defenders_in_box = averageDefendersInTheBoxPerGameByDF(blown)
stats.ttest_ind(held_defenders_in_box, blown_defenders_in_box, equal_var=False)

#### Create a new DataFrame (plays2) of the sames games in final, but with the defensive plays from the winning team (at the beginning of final) that do not qualify to be represented in final

In [ ]:
antifinal_plays = plays.set_index(['gameId','playId'])
plays2 = plays.set_index(['gameId', 'playId'])
for i in final.index.get_level_values(0).unique():
    # drop all games that are represented in df=final
    antifinal_plays.drop(index=i,axis=0,level=0,inplace=True)
for i in antifinal_plays.index.get_level_values(0).unique():
    # drop all games represented in df=antifinal from plays2
    # this results in a df that has all the plays for games represented in df=final
    plays2.drop(index=i,axis=0,inplace=True)
for i in final.index.values:
    # drop (gameid,playid) tuples that are found in final (plays2 = every play not in df=final)
    plays2.drop(index=i,axis=0,inplace=True)
    
plays2.reset_index(inplace=True)
# add column w the team that won to df
plays2['teamWin'] = plays2['gameId'].apply(tylertoW)

# make a df of all unique gameId,possessionTeam combinations from final
# this gets us the defensive possesions of the team that was winning from final
unique_gamepos = list(dict.fromkeys(list(zip(final.reset_index()['gameId'], final['possessionTeam']))))
unique_gameposdf = pd.DataFrame(unique_gamepos).set_index(0)
for i in np.arange(0,len(plays2)):
    # drop the row if the team on defense is not the same as the one in final 
    if plays2.loc[i]['possessionTeam'] != unique_gameposdf.loc[plays2.loc[i]['gameId']].values[0]:
        plays2.drop(index=i,axis=0,inplace=True)
plays2.reset_index(drop=True,inplace=True)

#### Add a column (heldOrBlown) to plays2

In [ ]:
# make a df of all unique gameId,held/blown combinations from final
gameid_heldblown = pd.DataFrame(list(dict.fromkeys(list(zip(final.reset_index()['gameId'], final['heldOrBlown']))))).set_index(0)

def gameidtoHB(i):
    """input a gameId and return whether the leading team held or blew the lead"""
    return gameid_heldblown.loc[i].values[0]

plays2['heldOrBlown'] = plays2['gameId'].apply(gameidtoHB)

#### Find the average number of pass rushers for plays2 split by held or blown lead games

In [ ]:
# get df's only where teams held or blew the lead
held2 = plays2[plays2['heldOrBlown'] == 'held'].reset_index()
blown2 = plays2[plays2['heldOrBlown'] == 'blown'].reset_index()

# use percentOfPassRushersByDataFrame function
held_lead_pass_rushers2 = pd.Series(percentOfPassRushersByDataFrame(held2, plays2))
blown_lead_pass_rushers2 = pd.Series(percentOfPassRushersByDataFrame(blown2, plays2))

# get average # of passrushers for teams that held the lead prior to a lead with under 4 min to go
average_held_lead_pass_rushers2 = 0
for i in range(0, 8):
    average_held_lead_pass_rushers2 = average_held_lead_pass_rushers2 + (held_lead_pass_rushers2[i] * i)

# get average # of passrushers for teams that blew the lead prior to a lead with under 4 min to go
average_blown_lead_pass_rushers2 = 0
for i in range(0, 8):
    average_blown_lead_pass_rushers2 = average_blown_lead_pass_rushers2 + (blown_lead_pass_rushers2[i] * i)
print(average_held_lead_pass_rushers2, average_blown_lead_pass_rushers2)